In [ ]:
pip install -e . -q

In [ ]:
pip uninstall sql_nameguard -y

In [ ]:
pip show sql_nameguard

In [ ]:
import os
from sql_nameguard.llm_suggest import LLMSuggester
from sql_nameguard.analyze import SQLAnalyzer
from dotenv import load_dotenv
load_dotenv()

In [ ]:
complex_sql = """
WITH revenue_cte AS (
    SELECT 
        c.id, 
        sum(o.amount) as total_rev
    FROM customers c
    JOIN orders o ON c.id = o.customer_id
    GROUP BY 1
),
risky_users AS (
    SELECT 
        id 
    FROM revenue_cte r
    WHERE r.total_rev > 10000 
      AND (CASE WHEN r.total_rev > 50000 THEN 1 ELSE 0 END) = 1
)
SELECT 
    t1.id, 
    t1.total_rev
FROM revenue_cte t1
LEFT JOIN (
    SELECT user_id, count(*) as c FROM logs GROUP BY 1
) t2 ON t1.id = t2.user_id
WHERE t1.id IN (SELECT id FROM risky_users)
"""

In [ ]:
analyzer = SQLAnalyzer()

aliases = analyzer.analyze(complex_sql, log_warnings=False)

In [ ]:
aliases

In [ ]:
poor_aliases = [alias for alias in aliases if alias['similarity'] < 0.4]

In [ ]:
poor_aliases

In [ ]:
model_cases = [
    {
        "provider": "openai",
        "model": "gpt-5-mini",
        "api_key": os.getenv("OPENAI_API_KEY")
    },
    {
        "provider": "anthropic",
        "model": "claude-haiku-4-5",
        "api_key": os.getenv("ANTHROPIC_API_KEY")
    },
    {   
        "provider": "google",
        "model": "gemini-2.5-flash-lite",
        "api_key": os.getenv("GOOGLE_API_KEY")
    }
]

In [ ]:
model_responses = {}
for case in model_cases:
    if case['api_key'] is None:
        raise ValueError(f"API key for {case['provider']} is not set.")
    api_key = case['api_key']
    provider = case['provider']
    model = case['model']

    print(f"## {provider}/{model} ############################")
    suggester = LLMSuggester(provider, model, api_key)

    responses = []
    for alias in poor_aliases:
        print(f'Selecting alternates for alias: {alias["alias"]}...')
        response = suggester.suggest_aliases(alias_type=alias['type'], code=alias['code'])
        responses.append(response)
    model_responses[f"{provider}_{model}"] = responses

In [ ]:
model_responses

In [ ]:
from sql_nameguard.llm_request import Message

test_message = Message.text('user', 'This is a test message.')

In [ ]:
test_message.parts[0].text

In [ ]:
from sql_nameguard.llm_call import call_llm, call_openai
from sql_nameguard.llm_request import LLMRequest

openai_case = model_cases[0]

request = LLMRequest(openai_case['provider'], openai_case['model'], [Message.text('user', 'What is the capital of France?')], max_tokens=1000, temperature=1)

response = call_llm(openai_case['provider'], request, openai_case['api_key'])
print(response)

In [ ]:
response

In [ ]:
antropic_case = model_cases[2]

anthropic_request = LLMRequest(antropic_case['provider'], antropic_case['model'], [Message.text('user', 'What is the capital of France?')], max_tokens=100, temperature=1)

anthropic_response = call_llm(antropic_case['provider'], anthropic_request, antropic_case['api_key'])
print(anthropic_response)